In [ ]:
from pyspark.sql import SparkSession
import os

In [ ]:
local=False
if local:
    spark=SparkSession.builder.master("local[4]") \
                  .appName("aida_poc_etl").getOrCreate()
else:
    spark=SparkSession.builder \
                      .master("k8s://https://kubernetes.default.svc:443") \
                      .appName("aida_poc_etl") \
                      .config("spark.kubernetes.container.image",os.environ["IMAGE_NAME"]) \
                      .config("spark.kubernetes.authenticate.driver.serviceAccountName",os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
                      .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
                      .config("spark.executor.instances", "10") \
                      .config("spark.executor.memory","16g") \
                      .config("spark.driver.memory","32g") \
                      .config('spark.jars.packages','org.postgresql:postgresql:42.2.24') \
                      .enableHiveSupport() \
                      .getOrCreate()
    
def set_log_level(spark_session,log_level:str):
    logger = spark_session.sparkContext._jvm.org.apache.log4j
    if log_level=="INFO":
        logger_level = logger.Level.INFO
    elif log_level=="WARN":
        logger_level = logger.Level.WARN
    elif log_level=="ERROR":
        logger_level = logger.Level.ERROR
    else:
        raise ValueError("The log_level must be INFO, WARN or ERROR")
    logger.LogManager.getLogger("org").setLevel(logger_level)
    logger.LogManager.getLogger("akka").setLevel(logger_level)
    
set_log_level(spark,"ERROR")

In [ ]:
work_dir="s3a://projet-poc-aida/rp"
parquet_file_name="individus_snappy_parquet"
data_path=f"{work_dir}/{parquet_file_name}"

# Step 1: Prepare source dataframe

Use spark context to read a parquet file and return a data frame 

In [ ]:
df_parquet=spark.read.parquet(data_path)

In [ ]:
df_parquet.show(5)

# Step2: Create a table in hive metastore

Use the spark dataframe to create a hive table in the hive metastore. So we can reuse it for later. 


In [ ]:
table_name="pengfei_test"

In [ ]:
schema_str = ', '.join([' '.join(x) for x in df_parquet.dtypes])

spark.sql(f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {table_name}
({schema_str})
STORED as parquet LOCATION '{data_path}'
""")


In [ ]:
spark.sql('show tables;').show()

Now your hive table has been created. In the backgroud, if you enabled the listener, the metadata of this hive table will be uploaded to our [data catalog](https://atlas.lab.sspcloud.fr/index.html#!/search). So you can find all your hive table easily even you don't have notebook anymore.

You can try to use the search engine of our [data catalog](https://atlas.lab.sspcloud.fr/index.html#!/search) to find your table. 

In [ ]:
spark.sql(f"""SELECT * FROM {table_name} limit 5""").show()

# Step 3. Drop table 

You can delete your table if you don't need it anymore. You will notice the metadata of the deleted table are `removed` from the data catalog too.  

In [ ]:
spark.sql(f"""drop table if exists {table_name}""").show()
spark.sql('show tables;').show()